# Explore Dynamic Dataset

Make lists of keys needed:

In [ ]:
datasets = ['21067-8-9', '22285-2-10', '22279-5-15', '22620-14-9', '22667-2-13']

keys = []
for d in datasets:
    animal, session, scan = d.split('-')
    keys.append({
     'animal_id': animal,
     'session': session,
     'scan_idx': scan,
     'preproc_id': 0
    })

Here I demonstrate how to fill tables necessary for dynamic dataset generation. 

In [ ]:
from neuro_data.movies import data_schemas, configs, stats

from neuro_data.movies.data_schemas import *
meso = dj.create_virtual_module('meso', 'pipeline_meso')
import datajoint as dj


import numpy as np

First thing first, make sure to insert your desired sessions into `MovieScanCandidate` table. Best would be to fill the `keys` list of dict like above and use the following line to insert.

In [ ]:
data_schemas.MovieScanCandidate().insert(fuse.ScanDone & keys, ignore_extra_fields=True, skip_duplicates=True)

Verify that your desired candidates are actually inserted

Now simply execute the tables below **in the following order**

In [ ]:
data_schemas.MovieScan.populate(keys, display_progress=True)

In [ ]:
data_schemas.ConditionTier.populate(keys, display_progress=True)

In [ ]:
data_schemas.MovieClips.populate(keys, display_progress=True)

In [ ]:
data_schemas.InputResponse.populate(keys, display_progress=True)

In [ ]:
data_schemas.Eye.populate(keys)

In [ ]:
data_schemas.Eye2.populate(keys)

In [ ]:
data_schemas.Treadmill.populate(keys)

In [ ]:
anatomy.AreaMembership() & keys

In [ ]:
anatomy.LayerMembership() & keys

If you know that they are from same area/layer, then you can use the following to manually fill the area and layer memberhsip table

In [ ]:
units = (meso.ScanSet.Unit & keys).fetch('KEY')

for u in units:
    u['brain_area'] = 'V1'
    u['layer'] = 'L2/3'

anatomy.AreaMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)

anatomy.LayerMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)

## To generate new entry into MovieMultiDataset

Run the following specifying the key, one at a time. **CAUTION**: it can also create a group with more than one key in it at a time, so carefully read the prompt! Please also be sure to give a meaningful description of the data group.

Register each key entry separtely

In [ ]:
for key in keys:
    name = '{animal_id}-{session}-{scan_idx}'.format(**key)
    MovieMultiDataset().add_entry(name, key)

## Generate HDF5 files

To trigger the generation of HDF5 files, run the following, but only upon ensuring that you have all above tables filled!

In [ ]:
for key in keys:
    key['preproc_id'] = 0
    #print('Generating HDF5 file for {}'.format(key))
    filename = data_schemas.InputResponse().get_filename(key)
    #print('Data for {} can be found in {}'.format(key, filename))
    print(filename)

## Fill out the Oracle score

In [ ]:
data_hash = dict(data_hash = "ecb7c24fafd19503a2eef756ac4a24a4")

In [ ]:
configs.DataConfig.AreaLayer & data_hash

In [ ]:
stats.Oracle.populate(MovieMultiDataset.Member & keys, data_hash)